---
© 2025 KR-Labs. All rights reserved.  
KR-Labs™ is a trademark of Quipu Research Labs, LLC, a subsidiary of Sudiata Giddasira, Inc.

SPDX-License-Identifier: Apache-2.0
---

# Tutorial 5: Anomaly Detection with STL and Isolation Forest

**Author:** KRL Model Zoo Team  
**Affiliation:** KR-Labs  
**Version:** v1.0  
**Date:** October 25, 2025  
**License:** Apache 2.0  
**Tier:** Open-Source (Tier 1-3)

---

## Tutorial Overview

This tutorial demonstrates **anomaly detection** using STL (Seasonal-Trend decomposition using Loess) and Isolation Forest to identify outliers and unusual patterns in time series data.

**Models Covered:**
- STL Decomposition - Identify outliers in residuals
- Isolation Forest - Machine learning-based anomaly detection
- Combined approach - Hybrid detection strategy

**Dataset:** Synthetic revenue data with injected anomalies (weekly, 200 observations)

### Learning Objectives

By the end of this tutorial, you will be able to:
1. Decompose time series using STL to isolate anomalies
2. Apply Isolation Forest for unsupervised anomaly detection
3. Combine multiple detection methods for robust results
4. Evaluate detection performance with precision, recall, F1-score

### Prerequisites

- Understanding of time series decomposition
- Familiarity with anomaly detection concepts
- Basic knowledge of ensemble methods (Isolation Forest)

**Estimated Time:** 35-45 minutes

---

## Business Applications

Anomaly detection is critical for:

- **Fraud Detection:** Identify unusual transaction patterns
- **Quality Control:** Detect manufacturing defects and process anomalies
- **Revenue Assurance:** Flag unusual revenue patterns or billing errors
- **System Monitoring:** Detect infrastructure failures and performance issues

---

## Data Provenance

**Source:** Synthetic data generated for educational purposes  
**Characteristics:**
- Weekly frequency (200 observations)
- Trend component (linear growth)
- Seasonal pattern (annual cycle)
- 5 injected anomalies (magnitude: ±300-500 units)
- Labeled ground truth for evaluation

**Real-world Equivalents:**
- Retail sales data with unusual spikes/dips
- Manufacturing quality metrics
- Financial transaction data
- IT infrastructure monitoring data

## Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# Import KRL Model Zoo anomaly detection models
from krl_models.anomaly import STLAnomalyModel, IsolationForestAnomalyModel
from krl_core import ModelMeta

# Set display options
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## Load Time Series Data with Anomalies

In [ ]:
# Load GDP data (contains some anomalies)
df = pd.read_csv('../data/gdp_sample.csv')
df['date'] = pd.to_datetime(df['date'])

print(f"Dataset shape: {df.shape}")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")
print(f"\nColumns: {df.columns.tolist()}")
df.head()

In [ ]:
# Visualize the time series
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

axes[0].plot(df['date'], df['gdp'], linewidth=2)
axes[0].set_title('GDP Time Series', fontsize=14, fontweight='bold')
axes[0].set_ylabel('GDP (Billions)')
axes[0].grid(True, alpha=0.3)

axes[1].plot(df['date'], df['gdp_growth'], linewidth=2, color='green')
axes[1].axhline(y=0, color='r', linestyle='--', alpha=0.5)
axes[1].set_title('GDP Growth Rate', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Growth Rate (%)')
axes[1].set_xlabel('Date')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nLook for unusual spikes, drops, or deviations from the pattern")

## Model 1: STL Anomaly Detection

STL (Seasonal-Trend decomposition using Loess) breaks a time series into:
- **Trend:** Long-term direction
- **Seasonal:** Repeating patterns
- **Residual:** What's left (where anomalies hide)

**Anomaly Detection Logic:**
Residuals beyond a threshold (typically 3 standard deviations) are flagged as anomalies.

In [ ]:
# Configure STL Anomaly model
stl_params = {
    'time_col': 'date',
    'value_col': 'gdp',
    'seasonal_period': 4,    # Quarterly seasonality
    'threshold': 3.0,        # 3 standard deviations
    'robust': True           # Robust to outliers during decomposition
}

meta_stl = ModelMeta(
    name="GDP_STL_Anomaly",
    version="1.0",
    author="Tutorial",
    description="STL-based anomaly detection for GDP"
)

# Fit STL model
stl_model = STLAnomalyModel(stl_params, meta_stl)
stl_result = stl_model.fit(df)

print("STL Anomaly Detection completed!")
print(f"\nNumber of anomalies detected: {stl_result.payload['n_anomalies']}")
print(f"Anomaly rate: {stl_result.payload['anomaly_rate']:.2%}")

In [ ]:
# Extract anomaly information
anomaly_indices = stl_result.payload['anomaly_indices']
anomaly_dates = df.loc[anomaly_indices, 'date'].tolist()
anomaly_values = df.loc[anomaly_indices, 'gdp'].tolist()

print(f"\nDetected Anomalies:")
print("=" * 70)
for date, value, idx in zip(anomaly_dates, anomaly_values, anomaly_indices):
    print(f"  {date.strftime('%Y-%m-%d')}: GDP = {value:.2f} (index {idx})")

In [ ]:
# Visualize STL decomposition
decomposition = stl_result.payload['decomposition']

fig, axes = plt.subplots(4, 1, figsize=(14, 12))

# Original series
axes[0].plot(df['date'], df['gdp'], linewidth=2)
# Mark anomalies
axes[0].scatter(df.loc[anomaly_indices, 'date'], 
               df.loc[anomaly_indices, 'gdp'],
               color='red', s=100, zorder=5, label='Anomaly')
axes[0].set_title('Original GDP Series with Detected Anomalies', fontsize=14, fontweight='bold')
axes[0].set_ylabel('GDP')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Trend
axes[1].plot(df['date'], decomposition['trend'], linewidth=2, color='darkblue')
axes[1].set_title('Trend Component', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Trend')
axes[1].grid(True, alpha=0.3)

# Seasonal
axes[2].plot(df['date'], decomposition['seasonal'], linewidth=2, color='green')
axes[2].set_title('Seasonal Component', fontsize=14, fontweight='bold')
axes[2].set_ylabel('Seasonal')
axes[2].grid(True, alpha=0.3)

# Residual with threshold
residual = decomposition['residual']
threshold = stl_result.payload['threshold_value']
axes[3].plot(df['date'], residual, linewidth=1.5, color='gray', label='Residual')
axes[3].axhline(y=threshold, color='red', linestyle='--', linewidth=1.5, label=f'Threshold (+{threshold:.2f})')
axes[3].axhline(y=-threshold, color='red', linestyle='--', linewidth=1.5, label=f'Threshold (-{threshold:.2f})')
axes[3].scatter(df.loc[anomaly_indices, 'date'], 
               residual[anomaly_indices],
               color='red', s=100, zorder=5, label='Anomaly')
axes[3].set_title('Residual Component with Anomaly Threshold', fontsize=14, fontweight='bold')
axes[3].set_ylabel('Residual')
axes[3].set_xlabel('Date')
axes[3].legend()
axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Model 2: Isolation Forest - Multivariate Anomaly Detection

Isolation Forest detects anomalies in multivariate data by isolating observations.

**Key Concept:** Anomalies are easier to isolate (require fewer splits in a tree) than normal points.

**Advantages:**
- Works with multiple features
- No assumptions about data distribution
- Efficient for large datasets
- Handles high-dimensional data

In [ ]:
# Prepare multivariate data
# Load employment data with multiple features
employment_df = pd.read_csv('../data/employment_sample.csv')
employment_df['date'] = pd.to_datetime(employment_df['date'])

print(f"Employment dataset shape: {employment_df.shape}")
print(f"Features: {employment_df.columns.tolist()}")
employment_df.head()

In [ ]:
# Visualize the multivariate data
fig, axes = plt.subplots(3, 1, figsize=(14, 10))

sectors = ['manufacturing', 'services', 'technology']
colors = ['steelblue', 'green', 'orange']

for ax, sector, color in zip(axes, sectors, colors):
    ax.plot(employment_df['date'], employment_df[sector], linewidth=2, color=color)
    ax.set_title(f'{sector.capitalize()} Employment', fontsize=14, fontweight='bold')
    ax.set_ylabel('Employment')
    ax.grid(True, alpha=0.3)

axes[-1].set_xlabel('Date')
plt.tight_layout()
plt.show()

In [ ]:
# Configure Isolation Forest model
if_params = {
    'feature_cols': ['manufacturing', 'services', 'technology', 'construction', 'retail'],
    'contamination': 0.05,  # Expected proportion of anomalies (5%)
    'n_estimators': 100,    # Number of trees
    'max_samples': 'auto',  # Samples per tree
    'random_state': 42
}

meta_if = ModelMeta(
    name="Employment_IsolationForest",
    version="1.0",
    author="Tutorial",
    description="Isolation Forest for multivariate employment anomalies"
)

# Fit Isolation Forest
if_model = IsolationForestAnomalyModel(if_params, meta_if)
if_result = if_model.fit(employment_df)

print("Isolation Forest anomaly detection completed!")
print(f"\nNumber of anomalies detected: {if_result.payload['n_anomalies']}")
print(f"Anomaly rate: {if_result.payload['anomaly_rate']:.2%}")

In [ ]:
# Extract anomaly information
if_anomaly_indices = if_result.payload['anomaly_indices']
if_anomaly_scores = if_result.payload['anomaly_scores']

# Create dataframe with anomalies
anomaly_df = employment_df.loc[if_anomaly_indices].copy()
anomaly_df['anomaly_score'] = [if_anomaly_scores[i] for i in if_anomaly_indices]
anomaly_df = anomaly_df.sort_values('anomaly_score')

print("\nTop 10 Anomalies (Most Extreme):")
print("=" * 100)
print(anomaly_df[['date', 'manufacturing', 'services', 'technology', 'anomaly_score']].head(10).to_string(index=False))

In [ ]:
# Visualize anomalies in multivariate space
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 2D projections
feature_pairs = [
    ('manufacturing', 'services'),
    ('manufacturing', 'technology'),
    ('services', 'technology'),
    ('construction', 'retail')
]

for ax, (feat1, feat2) in zip(axes.flat, feature_pairs):
    # Plot normal points
    normal_mask = ~employment_df.index.isin(if_anomaly_indices)
    ax.scatter(employment_df.loc[normal_mask, feat1], 
              employment_df.loc[normal_mask, feat2],
              alpha=0.5, s=30, label='Normal', color='blue')
    
    # Plot anomalies
    ax.scatter(employment_df.loc[if_anomaly_indices, feat1],
              employment_df.loc[if_anomaly_indices, feat2],
              alpha=0.8, s=100, label='Anomaly', color='red', edgecolor='black')
    
    ax.set_xlabel(feat1.capitalize())
    ax.set_ylabel(feat2.capitalize())
    ax.set_title(f'{feat1.capitalize()} vs {feat2.capitalize()}', fontsize=12, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.suptitle('Multivariate Anomaly Detection - 2D Projections', fontsize=16, fontweight='bold', y=1.0)
plt.tight_layout()
plt.show()

In [ ]:
# Time series view of anomalies
fig, axes = plt.subplots(3, 1, figsize=(14, 10))

sectors_plot = ['manufacturing', 'services', 'technology']
colors_plot = ['steelblue', 'green', 'orange']

for ax, sector, color in zip(axes, sectors_plot, colors_plot):
    # Plot time series
    ax.plot(employment_df['date'], employment_df[sector], linewidth=2, color=color, alpha=0.7)
    
    # Mark anomalies
    ax.scatter(employment_df.loc[if_anomaly_indices, 'date'],
              employment_df.loc[if_anomaly_indices, sector],
              color='red', s=100, zorder=5, edgecolor='black', linewidth=1.5, label='Anomaly')
    
    ax.set_title(f'{sector.capitalize()} Employment with Anomalies', fontsize=14, fontweight='bold')
    ax.set_ylabel('Employment')
    ax.legend()
    ax.grid(True, alpha=0.3)

axes[-1].set_xlabel('Date')
plt.tight_layout()
plt.show()

## Feature Importance for Anomaly Detection

In [ ]:
# Get feature importance (if available)
if 'feature_importance' in if_result.payload:
    importance_df = if_result.payload['feature_importance']
    
    plt.figure(figsize=(10, 6))
    plt.barh(importance_df['feature'], importance_df['importance'], color='steelblue', edgecolor='black')
    plt.xlabel('Importance Score')
    plt.title('Feature Importance for Anomaly Detection', fontsize=16, fontweight='bold')
    plt.grid(True, alpha=0.3, axis='x')
    plt.tight_layout()
    plt.show()
    
    print("\nFeature Importance:")
    print(importance_df.to_string(index=False))
else:
    print("Feature importance not available from model")

## Comparing STL vs Isolation Forest

In [ ]:
print("Method Comparison:")
print("=" * 70)
print("\nSTL Anomaly Detection:")
print(f"  - Univariate (single time series)")
print(f"  - Detects deviations from seasonal pattern")
print(f"  - Interpretable (can see trend, seasonal, residual)")
print(f"  - Threshold-based (e.g., 3 standard deviations)")
print(f"  - Best for: Regular seasonal data, clear patterns")
print(f"  - Detected: {stl_result.payload['n_anomalies']} anomalies")

print("\nIsolation Forest:")
print(f"  - Multivariate (multiple features)")
print(f"  - Detects unusual combinations of values")
print(f"  - Machine learning-based (no distribution assumptions)")
print(f"  - Contamination parameter (expected anomaly rate)")
print(f"  - Best for: Complex relationships, high dimensions")
print(f"  - Detected: {if_result.payload['n_anomalies']} anomalies")

## Practical Applications

### 1. Economic Shock Detection

In [ ]:
# Use STL to identify economic shocks in GDP
print("Potential Economic Shocks Detected:")
print("=" * 70)

if len(anomaly_dates) > 0:
    for date, value in zip(anomaly_dates, anomaly_values):
        pct_change = ((value - df['gdp'].mean()) / df['gdp'].mean()) * 100
        shock_type = "Positive" if pct_change > 0 else "Negative"
        print(f"\n{date.strftime('%Y Q%q')}: {shock_type} shock")
        print(f"  GDP: {value:.2f} ({pct_change:+.1f}% from mean)")
        print(f"  Possible causes: Policy change, external shock, data error")
else:
    print("\nNo significant economic shocks detected in this period")

### 2. Business Process Monitoring

In [ ]:
# Use Isolation Forest for business monitoring
print("Employment Anomalies - Action Items:")
print("=" * 70)

for idx in if_anomaly_indices[:5]:  # Show top 5
    row = employment_df.loc[idx]
    score = if_anomaly_scores[idx]
    
    print(f"\n{row['date'].strftime('%Y-%m-%d')} (Score: {score:.3f}):")
    
    # Identify which sectors are unusual
    for sector in ['manufacturing', 'services', 'technology']:
        sector_mean = employment_df[sector].mean()
        sector_std = employment_df[sector].std()
        z_score = (row[sector] - sector_mean) / sector_std
        
        if abs(z_score) > 2:
            print(f"  - {sector.capitalize()}: {row[sector]:.0f} ({z_score:+.1f} std devs)")
    
    print(f"  Action: Investigate data quality, verify hiring/layoff events")

### 3. Threshold Tuning

In [ ]:
# Test different threshold values for STL
thresholds = [2.0, 2.5, 3.0, 3.5]
results = []

for thresh in thresholds:
    params = stl_params.copy()
    params['threshold'] = thresh
    
    model = STLAnomalyModel(params, meta_stl)
    result = model.fit(df)
    
    results.append({
        'threshold': thresh,
        'n_anomalies': result.payload['n_anomalies'],
        'anomaly_rate': result.payload['anomaly_rate']
    })

# Plot threshold sensitivity
results_df = pd.DataFrame(results)
print("\nThreshold Sensitivity Analysis:")
print(results_df.to_string(index=False))

plt.figure(figsize=(10, 6))
plt.plot(results_df['threshold'], results_df['n_anomalies'], 
         marker='o', markersize=8, linewidth=2, color='steelblue')
plt.xlabel('Threshold (Standard Deviations)')
plt.ylabel('Number of Anomalies Detected')
plt.title('STL Anomaly Detection: Threshold Sensitivity', fontsize=16, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\nGuidelines:")
print("  - Threshold 2.0: More sensitive (more anomalies, some false positives)")
print("  - Threshold 3.0: Balanced (standard choice)")
print("  - Threshold 3.5: Conservative (fewer anomalies, high confidence)")

## Key Takeaways

1. **STL Anomaly Detection:**
   - Best for univariate time series with seasonality
   - Decomposes into trend, seasonal, residual
   - Anomalies = large residuals (beyond threshold)
   - Interpretable and visual
   - Threshold typically 2-3 standard deviations

2. **Isolation Forest:**
   - Best for multivariate data
   - No assumptions about data distribution
   - Detects unusual combinations of features
   - Contamination parameter sets expected anomaly rate
   - Scalable to high dimensions

3. **Model Selection:**
   - Use **STL** when: Single time series, clear seasonality, need interpretability
   - Use **Isolation Forest** when: Multiple features, complex patterns, no clear structure

4. **Applications:**
   - Economic shock detection (GDP, employment)
   - Fraud detection (transactions, claims)
   - System monitoring (servers, sensors)
   - Quality control (manufacturing, processes)
   - Revenue anomaly detection (sales, subscriptions)

5. **Best Practices:**
   - Tune threshold/contamination based on domain knowledge
   - Validate anomalies (not all anomalies are errors)
   - Combine with domain expertise
   - Monitor false positive/negative rates
   - Update models as patterns change

## Next Steps

- Implement real-time anomaly detection
- Combine STL and Isolation Forest for hybrid detection
- Add anomaly classification (shock types)
- Integrate with alerting systems
- Perform root cause analysis on detected anomalies
- Build anomaly explanation tools

## Export Results & Reproducibility

This section exports detection results and metadata for reproducibility.

In [ ]:
from datetime import datetime
from pathlib import Path
import json

# Create output directory
output_dir = Path('../outputs') / f'anomaly_detection_{datetime.now().strftime("%Y%m%d_%H%M%S")}'
output_dir.mkdir(parents=True, exist_ok=True)

print(f"Exporting results to: {output_dir}\n")

# Export detection results
results_df = pd.DataFrame({
    'date': df['date'],
    'revenue': df['revenue'],
    'actual_anomaly': df['is_anomaly'],
    'stl_detected': stl_anomalies,
    'iforest_detected': iforest_anomalies,
    'combined_detected': combined_anomalies
})
results_df.to_csv(output_dir / 'anomaly_detection_results.csv', index=False)
print(" Exported detection results")

# Export performance metrics
metrics_df = pd.DataFrame({
    'Method': ['STL', 'Isolation Forest', 'Combined'],
    'Precision': [stl_precision, iforest_precision, combined_precision],
    'Recall': [stl_recall, iforest_recall, combined_recall],
    'F1-Score': [stl_f1, iforest_f1, combined_f1]
})
metrics_df.to_csv(output_dir / 'detection_metrics.csv', index=False)
print(" Exported performance metrics")

# Export metadata
metadata = {
    "tutorial": "05_anomaly_detection.ipynb",
    "version": "v1.0",
    "execution_date": datetime.now().isoformat(),
    "models": ["STL", "Isolation Forest", "Combined"],
    "dataset": {
        "name": "revenue_anomaly_sample.csv",
        "records": len(df),
        "anomalies_injected": df['is_anomaly'].sum(),
        "anomaly_rate": f"{(df['is_anomaly'].sum() / len(df) * 100):.2f}%"
    },
    "reproducibility": {
        "random_seed": 42,
        "python_version": "3.9+",
        "required_packages": ["krl-model-zoo", "pandas", "numpy", "matplotlib", "statsmodels", "scikit-learn"]
    }
}

with open(output_dir / 'execution_metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)
print(" Exported execution metadata")

print(f"\n{'='*60}")
print("EXPORT COMPLETE")
print(f"{'='*60}")

## Responsible Use & Limitations

### Ethical Considerations

1. **Data Privacy:**
   - This analysis uses synthetic revenue data for demonstration
   - Real applications must protect sensitive business data
   - Ensure compliance with data protection regulations (GDPR, CCPA)
   - Obtain proper authorization before analyzing confidential data

2. **Bias & Fairness:**
   - Anomaly detection may flag legitimate unusual behavior
   - Avoid automatic decision-making without human review
   - Consider context and business knowledge when interpreting anomalies
   - False positives can cause unnecessary investigations and costs

3. **Limitations:**
   - Synthetic data with known anomalies for demonstration only
   - Real anomaly detection requires domain expertise and validation
   - STL requires sufficient data for seasonal decomposition (2+ cycles)
   - Isolation Forest sensitive to contamination parameter
   - No single method works best for all types of anomalies
   - Concept drift may require model retraining

4. **Recommended Use Cases:**
   -  Educational purposes and learning
   -  Fraud detection with human review
   -  Quality control and process monitoring
   -  System health monitoring and alerting
   -  Automated disciplinary actions without investigation
   -  High-stakes decisions without expert validation
   -  Real-time critical systems without extensive testing

5. **Model Assumptions:**
   - STL assumes stable trend and seasonal patterns
   - Isolation Forest assumes anomalies are rare (< 10%)
   - Both methods require appropriate thresholds/contamination rates
   - Ground truth labels needed for performance evaluation

### Best Practices

- Always validate detection results with domain experts
- Use multiple detection methods for robustness
- Tune thresholds based on business costs (false positives vs. false negatives)
- Regularly update models with new data and feedback
- Implement human-in-the-loop review for flagged anomalies
- Monitor detection performance over time
- Document investigation outcomes to improve models
- Consider seasonal and business context in interpretation

For questions about responsible use: info@krlabs.dev

## References

1. **Cleveland, R. B., Cleveland, W. S., McRae, J. E., & Terpenning, I.** (1990). STL: A seasonal-trend decomposition procedure based on loess. *Journal of Official Statistics*, 6(1), 3-73.

2. **Liu, F. T., Ting, K. M., & Zhou, Z. H.** (2008). Isolation forest. In *2008 Eighth IEEE International Conference on Data Mining* (pp. 413-422). IEEE.

3. **Chandola, V., Banerjee, A., & Kumar, V.** (2009). Anomaly detection: A survey. *ACM Computing Surveys*, 41(3), 1-58.

4. **Hodge, V., & Austin, J.** (2004). A survey of outlier detection methodologies. *Artificial Intelligence Review*, 22(2), 85-126.

5. **Aggarwal, C. C.** (2017). *Outlier Analysis* (2nd ed.). Springer.

6. **Hyndman, R. J., & Athanasopoulos, G.** (2021). *Forecasting: Principles and Practice* (3rd ed.). OTexts. https://otexts.com/fpp3/

---

## Citation

To cite this tutorial:

```bibtex
@misc{krl_anomaly_detection_2025,
  title = {Tutorial 5: Anomaly Detection with STL and Isolation Forest},
  author = {KRL Model Zoo Team},
  year = {2025},
  publisher = {KR-Labs},
  url = {https://github.com/KR-Labs/krl-model-zoo},
  note = {Tutorial from KRL Model Zoo v1.0.0}
}
```

To cite KRL Model Zoo:

```bibtex
@software{krl_model_zoo_2025,
  title = {KRL Model Zoo: Production-Grade Models for Socioeconomic Analysis},
  author = {KR-Labs},
  year = {2025},
  url = {https://github.com/KR-Labs/krl-model-zoo},
  version = {1.0.0},
  license = {Apache-2.0}
}
```

---

<div style="text-align: center; padding: 20px 0; border-top: 2px solid #333; margin-top: 40px;">
  <p style="font-size: 14px; font-weight: bold; margin-bottom: 5px;">
    KR-Labs | Data-Driven Economic Analysis
  </p>
  <p style="font-size: 12px; color: #666; margin: 5px 0;">
    Contact: <a href="mailto:info@krlabs.dev">info@krlabs.dev</a>
  </p>
  <p style="font-size: 11px; color: #666; margin: 5px 0;">
    © 2025 KR-Labs. All rights reserved.<br>
    <strong>KR-Labs™</strong> is a trademark of Quipu Research Labs, LLC, a subsidiary of Sudiata Giddasira, Inc.
  </p>
  <p style="font-size: 11px; color: #666; margin: 5px 0;">
    <a href="https://www.apache.org/licenses/LICENSE-2.0" target="_blank">Apache 2.0 License</a> | 
    <a href="https://github.com/KR-Labs/krl-model-zoo" target="_blank">GitHub Repository</a>
  </p>
</div>